In [2]:
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [3]:
file = "datasets/data.csv"
recipes_df = pd.read_csv(file)

In [4]:
new_file = "datasets/walmart.csv"
walmart_df = pd.read_csv(new_file)

In [5]:
#recipes_df.loc[(recipes_df["Main Ingredient"] == 'Eggs')]

In [6]:
# Create a proteins_df with the 20 main ingredients and make this a table in the database
df = recipes_df[["Main Ingredient"]].drop_duplicates()
proteins_df = df.reset_index(drop=True)
#proteins_df


In [7]:
# Need to save the proteins without spaces or capitalization so that our link to the Walmart store will work in SQL

proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Chicken breast":"Chickenbreast"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Cottage cheese":"Cottagecheese"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Greek yogurt":"Greekyogurt"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Lean beef":"Leanbeef"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Whey protein supplements":"Wheyproteinsupplements"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Ezekiel bread":"Ezekielbread"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Pumpkin seeds":"Pumpkinseeds"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Turkey breast":"Turkeybreast"},regex=True)
proteins_df["Main Ingredient"] = proteins_df["Main Ingredient"].replace({"Brussels sprouts":"Brusselssprouts"},regex=True)

proteins_df.rename(columns={"Main Ingredient":"Main_Ingredient"},inplace=True)
proteins_df.index.name = "ID"
proteins_df


,Main_Ingredient
ID,
0,Eggs
1,Almonds
2,Chickenbreast
3,Oats
4,Cottagecheese
5,Greekyogurt
6,Milk
7,Broccoli
8,Leanbeef


In [8]:
#walmart_df

In [9]:
# Trim walmart_df by removing "ingredients" and "urls", save as storeinfo_df. 
# Now it only shows the unique Walmart stores in the northeast and their location information
df = walmart_df[["name", "store_id", "phone", "postalCode", "address", "city", "state", "country"]].drop_duplicates()
storeinfo_df = df.reset_index(drop=True)
storeinfo_df

,name,store_id,phone,postalCode,address,city,state,country
0,Aberdeen Supercenter,1968,410-273-9200,21001,645 S Philadelphia Blvd,Aberdeen,MD,US
1,Abingdon Store,2009,410-569-9403,21009,401 Constant Friendship Blvd,Abingdon,MD,US
2,Allentown Supercenter,2641,610-530-1400,18106,1091 Millcreek Rd,Allentown,PA,US
3,Audubon Store,5047,856-310-1470,8106,130 Black Horse Pike,Audubon,NJ,US
4,Bayonne Supercenter,5867,201-620-6137,7002,500 Bayonne Crossing Way,Bayonne,NJ,US
...,...,...,...,...,...,...,...,...
144,Willow Grove Supercenter,3564,215-830-8370,19090,2101 Blair Mill Rd,Willow Grove,PA,US
145,Wilmington Store,5436,302-683-9312,19808,1251 Centerville Rd,Wilmington,DE,US
146,Woodbridge Store,5281,732-826-4652,7095,360 Us Highway 9 Route N,Woodbridge,NJ,US
147,Woodbury Supercenter,5476,856-686-0133,8096,820 Cooper St,Woodbury,NJ,US


In [11]:
connection = f"postgres:{password}@localhost:5432/ETL"
engine = create_engine(f'postgresql://{connection}')

In [12]:
engine.table_names()

['protein_list']

In [12]:
# Proteins table cleanup
protein_list_df = proteins_df.rename(columns={"Main Ingredient":"Main_Ingredient"},inplace=True)
protein_list_df

In [13]:
# Recipe Table cleanup
recipetable_df = recipes_df[["Main Ingredient","Recipe Name","Cook Time","Yield","Calories","URL"]]
recipetable_df.rename(columns={"Main Ingredient":"Main_Ingredient","Recipe Name":"Recipe_Name","Cook Time":"Cook_time","URL":"URL"},inplace=True)

recipetable_df.index.name = "ID"
recipetable_df.index += 1 
recipetable_df

,Main_Ingredient,Recipe_Name,Cook_time,Yield,Calories,URL
ID,,,,,,
1,Eggs,Scotch Eggs,25.0,4.0,3172.921085,http://www.seriouseats.com/recipes/2012/03/sco...
2,Eggs,Moonstruck eggs,24.0,2.0,786.910000,https://food52.com/recipes/518-moonstruck-eggs
3,Eggs,Eggs Benedict,60.0,4.0,2863.087500,http://www.bbc.co.uk/food/recipes/eggsbenedict...
4,Eggs,Deviled eggs,17.0,4.0,302.270000,http://honestcooking.com/deviled-eggs/
5,Eggs,Scrambled Eggs,15.0,2.0,675.763984,http://www.tastingtable.com/entry_detail/chefs...
...,...,...,...,...,...,...
939,Peanuts,Thai Cucumber Salad with Peanuts,20.0,4.0,935.830000,http://www.delish.com/cooking/recipe-ideas/rec...
940,Peanuts,Asian Chicken with Peanuts,45.0,4.0,1825.653876,http://www.myrecipes.com/recipe/asian-chicken-...
941,Peanuts,"Grilled Shrimp with Cilantro, Lime, and Peanuts",20.0,4.0,903.322406,http://www.marthastewart.com/339717/grilled-sh...


In [14]:
# Walmart Table cleanup
store_df = storeinfo_df[["store_id","name","address","postalCode","city","state"]]
store_df.rename(columns={"store_id":"Store_Id","name":"Store_Name","address":"Store_Address", "postalCode":"Store_Zipcode", "city":"Store_City","state":"Store_State"},inplace=True)

#store_df.set_index("Store_id",inplace=True)
store_df.index.name = "ID"
store_df

C:\Users\ramya\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Store_Id,Store_Name,Store_Address,Store_Zipcode,Store_City,Store_State
ID,,,,,,
0,1968,Aberdeen Supercenter,645 S Philadelphia Blvd,21001,Aberdeen,MD
1,2009,Abingdon Store,401 Constant Friendship Blvd,21009,Abingdon,MD
2,2641,Allentown Supercenter,1091 Millcreek Rd,18106,Allentown,PA
3,5047,Audubon Store,130 Black Horse Pike,8106,Audubon,NJ
4,5867,Bayonne Supercenter,500 Bayonne Crossing Way,7002,Bayonne,NJ
...,...,...,...,...,...,...
144,3564,Willow Grove Supercenter,2101 Blair Mill Rd,19090,Willow Grove,PA
145,5436,Wilmington Store,1251 Centerville Rd,19808,Wilmington,DE
146,5281,Woodbridge Store,360 Us Highway 9 Route N,7095,Woodbridge,NJ


In [16]:
# Load dataframes into database, Table: Protein_List
proteins_df.to_sql(name='Protein_List', con=engine, if_exists='append', index=True)

In [ ]:
# Load dataframes into database, Table: Recipes
recipetable_df.to_sql(name='Recipes', con=engine, if_exists='append', index=True)

In [ ]:
# Load dataframes into database, Table: Walmart
store_df.to_sql(name='Store', con=engine, if_exists='append', index=True)